In [1]:
import pandas as pd 
import pickle
import warnings
warnings.filterwarnings('ignore')
import numpy as np

from sklearn.metrics import classification_report
from sklearn import preprocessing
import torch
import torch.nn as nn
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

In [2]:
y_train = pd.read_csv("/Users/yoshithaakunuri/Documents/CSCI535/Project/Final/lexical/text_feature_extraction/Emotion Bert/embeddings/train_labels.csv")
y_test = pd.read_csv("/Users/yoshithaakunuri/Documents/CSCI535/Project/Final/lexical/text_feature_extraction/Emotion Bert/embeddings/test_labels.csv")


In [3]:
with open('/Users/yoshithaakunuri/Documents/CSCI535/Project/Final/lexical/text_feature_extraction/Emotion Bert/embeddings/train_bert_emo_embeddings_target_.pkl', 'rb') as f:
    x_train = pickle.load(f, encoding='latin1')

with open('/Users/yoshithaakunuri/Documents/CSCI535/Project/Final/lexical/text_feature_extraction/Emotion Bert/embeddings/test_bert_emo_embeddings_target_.pkl', 'rb') as f:
    x_test = pickle.load(f, encoding='latin1')
    

In [4]:
x_train_vals = []
for sample in x_train["embeddings"]:
    x_train_vals.append(torch.stack(sample))
    
x_test_vals = []
for sample in x_test["embeddings"]:
    x_test_vals.append(torch.stack(sample))

x_train_df = pd.DataFrame({'embeddings':x_train_vals})
x_test_df = pd.DataFrame({'embeddings':x_test_vals})

x_train_df["sarcasm"] = y_train["sarcasm"]
x_test_df["sarcasm"] = y_test["sarcasm"]
x_train_df["sarcasm"] = x_train_df["sarcasm"].astype('int').to_numpy()
x_test_df["sarcasm"] = x_test_df["sarcasm"].astype('int').to_numpy()

In [5]:
class GRUTensorDataset(Dataset):
    def __init__(self, dataframe, speaker):
        self.data = dataframe
        self.speaker = speaker

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        if self.speaker:
            features = self.data.loc[index, 'embeddings']
            features = np.array(features)
            a=np.empty((512,1))
            a.fill(self.data.loc[index, 'speaker'])
            final_features = np.hstack((features, a))
            label = self.data.loc[index, 'sarcasm']
            return torch.from_numpy(final_features).float(), label
        else:
            features = self.data.loc[index, 'embeddings']
            features = np.array(features)
            label = self.data.loc[index, 'sarcasm']
            return torch.from_numpy(features).float(), label

    def __getindexlist__(self):
        return list(self.data.index.values)
    
class GRUNetSD(nn.Module):
    def __init__(self, input_dim, hidden_dim, 
                 output_dim, n_layers):
        super(GRUNetSD, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        
        self.gru = nn.GRU(input_dim, hidden_dim, 
                          n_layers, batch_first = True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, x, h):
        out, h = self.gru(x, h)
        out = self.softmax(self.fc(out[:,-1]))
        return out, h
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = weight.new(self.n_layers, batch_size, 
                            self.hidden_dim).zero_()
        return hidden
    
class GRUNetSID(nn.Module):
    def __init__(self, input_dim, hidden_dim, 
                 output_dim, n_layers):
        super(GRUNetSID, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        
        self.gru = nn.GRU(input_dim, hidden_dim, 
                          n_layers, batch_first = True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, x, h):
        out, h = self.gru(x, h)
        out = self.softmax(self.fc(out[:,-1]))
        return out, h
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = weight.new(self.n_layers, batch_size, 
                            self.hidden_dim).zero_()
        return hidden
    
def evaluateGRU(gru, review, size):
    hidden = gru.init_hidden(size)
    output, hidden = gru(review, hidden)
    return output

def categoryFromOutput(output):
    top_n, top_i = torch.max(output,dim=1)
    return top_i

def test_accuracy(gru, loader, size):
    actuals = []
    predictions = []
    for data, target in loader:
        output = evaluateGRU(gru, data, size)
        prediction_index = categoryFromOutput(output)
        predictions = prediction_index.tolist()
        actuals = target.tolist()
    return predictions, actuals
    
hidden_size = 50
output_size = 2
input_size_sid = 768
input_size_sd = 769
n_layers = 1

### Speaker Independent and Context Independent

In [6]:
gru_train_tensor = GRUTensorDataset(x_train_df[['embeddings', 'sarcasm']], False)
gru_test_tensor = GRUTensorDataset(x_test_df[['embeddings', 'sarcasm']], False)

num_of_workers = 0
batch_size = 31
valid_size = 0.1

train_indices = list(range(len(gru_train_tensor)))
np.random.shuffle(train_indices)

test_indices = list(range(len(gru_test_tensor)))
np.random.shuffle(test_indices)

train_loader = torch.utils.data.DataLoader(
    gru_train_tensor, 
    batch_size=batch_size, 
    drop_last = True,
    sampler=SubsetRandomSampler(train_indices)
)

test_loader = torch.utils.data.DataLoader(
    gru_test_tensor, 
    batch_size=batch_size, 
    drop_last = True,
    sampler=SubsetRandomSampler(test_indices)
)

In [7]:
gru = GRUNetSID(input_size_sid, hidden_size, output_size, n_layers)
print(gru)
criterion = nn.NLLLoss()
optimizer = torch.optim.Adam(gru.parameters(), lr=0.001)

GRUNetSID(
  (gru): GRU(768, 50, batch_first=True)
  (fc): Linear(in_features=50, out_features=2, bias=True)
  (softmax): LogSoftmax(dim=1)
)


In [8]:
n_epochs = 61
    
test_min_loss = np.inf

for epoch in range(n_epochs):
    torch.manual_seed(42)
    train_loss = 0.0
    test_loss = 0.0
    gru.train()
    for data, target in train_loader:
        h = gru.init_hidden(batch_size)
        optimizer.zero_grad()
        output, h = gru(data, h.data)
        loss = criterion(output, target.long())
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*data.size(0)

    gru.eval()
    for data, target in test_loader:
        if data.shape[1] < 44:
            continue
        h = gru.init_hidden(batch_size)
        output, h = gru(data, h.data)
        loss = criterion(output, target.long())
        test_loss += loss.item()*data.size(0)

    train_loss = train_loss / len(train_loader.dataset)
    test_loss = test_loss / len(test_loader.dataset)

    if(epoch%5 == 0):
        print("Epoch: " + str(epoch))
        test_loader_epoch = torch.utils.data.DataLoader(gru_test_tensor, batch_size=gru_test_tensor.__len__())
        predictions, actuals = test_accuracy(gru, test_loader_epoch, gru_test_tensor.__len__())
        print(pd.DataFrame(classification_report(actuals, predictions, output_dict=True)).T)

Epoch: 0
              precision    recall  f1-score     support
0              0.515695  0.950413  0.668605  121.000000
1              0.666667  0.100000  0.173913  120.000000
accuracy       0.526971  0.526971  0.526971    0.526971
macro avg      0.591181  0.525207  0.421259  241.000000
weighted avg   0.590868  0.526971  0.422285  241.000000
Epoch: 5
              precision    recall  f1-score     support
0              0.666667  0.479339  0.557692  121.000000
1              0.590909  0.758333  0.664234  120.000000
accuracy       0.618257  0.618257  0.618257    0.618257
macro avg      0.628788  0.618836  0.610963  241.000000
weighted avg   0.628945  0.618257  0.610742  241.000000
Epoch: 10
              precision    recall  f1-score     support
0              0.709091  0.322314  0.443182  121.000000
1              0.559140  0.866667  0.679739  120.000000
accuracy       0.593361  0.593361  0.593361    0.593361
macro avg      0.634115  0.594490  0.561460  241.000000
weighted avg   0.634

### Speaker Dependent and Context Independent

In [14]:
x_train_speakers = pd.read_csv("/Users/yoshithaakunuri/Documents/CSCI535/Project/Final/lexical/text_feature_extraction/Emotion Bert/embeddings/train_data.csv")
x_test_speakers = pd.read_csv("/Users/yoshithaakunuri/Documents/CSCI535/Project/Final/lexical/text_feature_extraction/Emotion Bert/embeddings/test_data.csv")
x_train_speakers

,target_,target_context,speaker
0,[CLS] I've been told it's a good way to move o...,[CLS] I've been told it's a good way to move o...,25
1,"[CLS] Yeah, sure. You slept with your husband....","[CLS] Yeah, sure. You slept with your husband....",1
2,[CLS] When are you coming home? [SEP],[CLS] When are you coming home? Okay. Alright....,16
3,[CLS] Riveting. [SEP],[CLS] Riveting. Bingo. Then I lifted the cushi...,0
4,"[CLS] No, this is just part of a daredevil gam...","[CLS] No, this is just part of a daredevil gam...",2
...,...,...,...
956,"[CLS] Oh, that's sweet, but today is all about...","[CLS] Oh, that's sweet, but today is all about...",7
957,[CLS] If you wanna put a label on it. [SEP],[CLS] If you wanna put a label on it. You mean...,24
958,[CLS] That you're an alcoholic? [SEP],[CLS] That you're an alcoholic? I realized som...,3
959,[CLS] All I see is a yellow smudge. [SEP],[CLS] All I see is a yellow smudge. Now go bac...,15


In [15]:
x_train_df["speaker"] = x_train_speakers["speaker"]
x_test_df["speaker"] = x_test_speakers["speaker"]
x_train_df

,embeddings,sarcasm,speaker
0,"[[tensor(0.4818), tensor(0.3962), tensor(-0.10...",0,25
1,"[[tensor(0.1864), tensor(-0.0127), tensor(-0.2...",0,1
2,"[[tensor(0.3204), tensor(0.1967), tensor(-0.41...",0,16
3,"[[tensor(-0.2869), tensor(0.3007), tensor(0.05...",1,0
4,"[[tensor(-0.0119), tensor(-0.0834), tensor(-0....",1,2
...,...,...,...
956,"[[tensor(-0.2093), tensor(-0.3899), tensor(0.0...",0,7
957,"[[tensor(0.0286), tensor(0.0007), tensor(-0.29...",1,24
958,"[[tensor(0.2307), tensor(1.1027), tensor(-0.85...",1,3
959,"[[tensor(-0.0889), tensor(0.5095), tensor(0.10...",0,15


In [16]:
gru_train_tensor = GRUTensorDataset(x_train_df[['embeddings', 'sarcasm', 'speaker']], True)
gru_test_tensor = GRUTensorDataset(x_test_df[['embeddings', 'sarcasm', 'speaker']], True)

num_of_workers = 0
batch_size = 31
valid_size = 0.1

train_indices = list(range(len(gru_train_tensor)))
np.random.shuffle(train_indices)

test_indices = list(range(len(gru_test_tensor)))
np.random.shuffle(test_indices)

train_loader = torch.utils.data.DataLoader(
    gru_train_tensor, 
    batch_size=batch_size, 
    drop_last = True,
    sampler=SubsetRandomSampler(train_indices)
)

test_loader = torch.utils.data.DataLoader(
    gru_test_tensor, 
    batch_size=batch_size, 
    drop_last = True,
    sampler=SubsetRandomSampler(test_indices)
)

In [17]:
gru = GRUNetSID(input_size_sd, hidden_size, output_size, n_layers)
print(gru)
criterion = nn.NLLLoss()
optimizer = torch.optim.Adam(gru.parameters(), lr=0.001)

GRUNetSID(
  (gru): GRU(769, 50, batch_first=True)
  (fc): Linear(in_features=50, out_features=2, bias=True)
  (softmax): LogSoftmax(dim=1)
)


In [18]:
n_epochs = 11
    
test_min_loss = np.inf

for epoch in range(n_epochs):
    torch.manual_seed(42)
    train_loss = 0.0
    test_loss = 0.0
    gru.train()
    for data, target in train_loader:
        h = gru.init_hidden(batch_size)
        optimizer.zero_grad()
        output, h = gru(data, h.data)
        loss = criterion(output, target.long())
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*data.size(0)

    gru.eval()
    for data, target in test_loader:
        if data.shape[1] < 44:
            continue
        h = gru.init_hidden(batch_size)
        output, h = gru(data, h.data)
        loss = criterion(output, target.long())
        test_loss += loss.item()*data.size(0)

    train_loss = train_loss / len(train_loader.dataset)
    test_loss = test_loss / len(test_loader.dataset)

    if(epoch%5 == 0):
        print("Epoch: " + str(epoch))
        test_loader_epoch = torch.utils.data.DataLoader(gru_test_tensor, batch_size=gru_test_tensor.__len__())
        predictions, actuals = test_accuracy(gru, test_loader_epoch, gru_test_tensor.__len__())
        print(pd.DataFrame(classification_report(actuals, predictions, output_dict=True)).T)

Epoch: 0
              precision    recall  f1-score     support
0              0.695122  0.471074  0.561576  121.000000
1              0.597484  0.791667  0.681004  120.000000
accuracy       0.630705  0.630705  0.630705    0.630705
macro avg      0.646303  0.631371  0.621290  241.000000
weighted avg   0.646506  0.630705  0.621042  241.000000
Epoch: 5
              precision    recall  f1-score     support
0              0.626866  0.694215  0.658824  121.000000
1              0.654206  0.583333  0.616740  120.000000
accuracy       0.639004  0.639004  0.639004    0.639004
macro avg      0.640536  0.638774  0.637782  241.000000
weighted avg   0.640479  0.639004  0.637869  241.000000
Epoch: 10
              precision    recall  f1-score     support
0              0.608108  0.743802  0.669145  121.000000
1              0.666667  0.516667  0.582160  120.000000
accuracy       0.630705  0.630705  0.630705    0.630705
macro avg      0.637387  0.630234  0.625652  241.000000
weighted avg   0.637